# Aminoacid Language Model

First make a language model, then make it into a classifier.

In [1]:
import torch
from pathlib import Path
import pandas as pd
import numpy as np

In the PyTorch docs: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

We can see that the hidden-to-gidden weight are saved in this manner:
>~LSTM.weight_hh_l[k] – the learnable hidden-hidden weights of the kth\text{k}^{th}kth layer (W_hi|W_hf|W_hg|W_ho), of shape (4*hidden_size, hidden_size)

We can use this to apply WeightDropout or DropConnect to the LSTM layers.
Also using https://pytorchnlp.readthedocs.io/en/latest/_modules/torchnlp/nn/weight_drop.html and FastAI docs https://github.com/fastai/fastai/blob/45376f13df04ddf72749be25ae8a6dff35859f68/fastai/text/models/awdlstm.py as inspiration.

## Load the Data

In [2]:
data_file = Path('/home/mees/Desktop/Machine_Learning/subcellular_location/data/processed/protein_data_2021-02-16.csv')
df = pd.read_csv(data_file, sep=';')
df.head()

,Sequence,Subcellular location [CC]
0,MTDTVFSNSSNRWMYPSDRPLQSNDKEQLQAGWSVHPGGQPDRQRK...,Membrane
1,MDTDSQRSHLSSFTMKLMDKFHSPKIKRTPSKKGKPAEVSVKIPEK...,Endosome
2,MEDSTSPKQEKENQEELGETRRPWEGKTAASPQYSEPESSEPLEAK...,Cytoplasm
3,MALPGARARGWAAAARAAQRRRRVENAGGSPSPEPAGRRAALYVHW...,Mitochondrion
4,MALLVDRVRGHWRIAAGLLFNLLVSICIVFLNKWIYVYHGFPNMSL...,pass membrane protein


We create a language model, the location is thus not used.

In [4]:
del df['Subcellular location [CC]']
df.head()

,Sequence
0,MTDTVFSNSSNRWMYPSDRPLQSNDKEQLQAGWSVHPGGQPDRQRK...
1,MDTDSQRSHLSSFTMKLMDKFHSPKIKRTPSKKGKPAEVSVKIPEK...
2,MEDSTSPKQEKENQEELGETRRPWEGKTAASPQYSEPESSEPLEAK...
3,MALPGARARGWAAAARAAQRRRRVENAGGSPSPEPAGRRAALYVHW...
4,MALLVDRVRGHWRIAAGLLFNLLVSICIVFLNKWIYVYHGFPNMSL...


## Tokenize the Data

In [7]:
# Set-up numpy generator for random numbers
random_number_generator = np.random.default_rng()
KMER_SIZE = 3

In [6]:
# Tokenize the protein sequence (or any sequence) in kmers.
def tokenize(protein_seqs, kmer_sz):
    kmers = set()
    # Loop over protein sequences
    for protein_seq in protein_seqs:
        # Loop over the whole sequence
        for i in range(len(protein_seq) - (kmer_sz - 1)):
            # Add kmers to the set, thus only unique kmers will remain
            kmers.add(protein_seq[i: i + kmer_sz])
            
    # Map kmers for one hot-encoding
    kmer_to_id = dict()
    id_to_kmer = dict()
    
    for ind, kmer in enumerate(kmers):
        kmer_to_id[kmer] = ind
        id_to_kmer[ind] = kmer
        
    vocab_sz = len(kmers)
    
    assert vocab_sz == len(kmer_to_id.keys())
    
    # Tokenize the protein sequence to integers
    tokenized = []
    for protein_seq in protein_seqs:
        sequence = []
        for i in  range(len(protein_seq) - (kmer_sz -1)):
            # Convert kmer to integer
            kmer = protein_seq[i: i + kmer_sz]
            sequence.append(kmer_to_id[kmer])
            
        tokenized.append(sequence)
            
    
    return tokenized, vocab_sz, kmer_to_id, id_to_kmer

In [8]:
# Tokenize the protein sequence
tokenized_seqs, vocab_sz, kmer_to_id, id_to_kmer = tokenize(df['Sequence'], KMER_SIZE)

In [9]:
vocab_sz

8071

In [11]:
tokenized_seqs[0][:10]

[2752, 1666, 6051, 5844, 6799, 1494, 3286, 3775, 4047, 902]

## Build a Dataset

In [12]:
class AminoSequenceDataset(torch.utils.data.Dataset):
    r""" A custom dataset for amino acid sequences.
    """
    def __init__(self, amino_sequences):
        self.amino_sequences = amino_sequences
        
    def __len__(self):
        return len(self.amino_sequences)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        sample = self.amino_sequences[idx]
        
        return torch.tensor(sample)

In [13]:
data = AminoSequenceDataset(tokenized_seqs)

In [15]:
data[0][:10]

tensor([2752, 1666, 6051, 5844, 6799, 1494, 3286, 3775, 4047,  902])

## Build the Language Model

In [16]:
class WeightDropout(torch.nn.Module):
    "Apply dropout to LSTM's hidden-hidden weights"
    
    def __init__(self, module, weight_p):
        super(WeightDropout, self).__init__()
        self.module = module
        self.weight_p = weight_p
        
        # Save the name of the layer weights in a list
        num_layers = module.num_layers
        layer_base_name = 'weight_hh_l'      
        self.layer_weights = [layer_base_name + str(i) for i in range(num_layers)]
        
        # Make a copy of the weights in weightname_raw
        for weight in self.layer_weights:
            w = getattr(self.module, weight)
            del module._parameters[weight]
            self.module.register_parameter(f'{weight}_raw', torch.nn.Parameter(w))
            
        def _setweights(self):
            "Apply dropout to the raw weights"
            for weight in self.layer_weights:
                raw_w = getattr(self, f'{weight}_raw')
                if self.training:
                    w = torch.nn.F(raw_w, p=self.weight_p)
                else:
                    w = raw_w.clone()
                setattr(self.module, weight, w)
                
        def forward(self, *args):
            self._setweights()
            return self.module(*args)   

In [17]:
emb_dim = 400 # Embeddding dimension
hid_sz = 1150 # Hidden size
num_layers = 3 # Number of LSTM layers stacked together

In [18]:
embed_p = 0.1 # Dropout probability on the embedding
hidden_p = 0.3 # Dropout probability on hidden-to-hidden weight matrices
input_p = 0.3 # Dropout probablity on the LSTM input between LSTMS

# This one still has to be implemented
#weight_p = 0.5 # Dropout probability on LSTM-to-LSTM weight matrices

In [19]:
model = torch.nn.Sequential(
    torch.nn.Embedding(vocab_sz, emb_dim),
    torch.nn.Dropout(p=0.1),
    WeightDropout(
        torch.nn.LSTM(input_size = emb_dim, hidden_size = hid_sz, num_layers = num_layers, dropout=input_p),
        hidden_p
    ),
    torch.nn.Linear(emb_dim, vocab_sz)
)

In [20]:
model

Sequential(
  (0): Embedding(8071, 400)
  (1): Dropout(p=0.1, inplace=False)
  (2): WeightDropout(
    (module): LSTM(400, 1150, num_layers=3, dropout=0.3)
  )
  (3): Linear(in_features=400, out_features=8071, bias=True)
)

## Train the Language Model

In [22]:
# Load the data 
data_loader = torch.utils.data.DataLoader(data, batch_size=1, shuffle=False)

In [33]:
learning_rate = 0.01
epochs = 1

In [34]:
# Pytorch stuff
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

In [35]:
loss_history = []  

In [45]:
for epoch in range(epochs):
    
    # Initialize loss at 0
    epoch_loss = 0.0
    
    print(f'Epoch: {str(epoch + 1)}')
    
    for i, data in enumerate(data_loader, 0):

        for j in enumerate(data[i]):
            inp = data[i]
            target = data[i + 1]
            
            optimizer.zero_grad()
            
            output = model(inp)
            loss = criterion(output, target)
            
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
    
        break
    
    loss_history.append(epoch_loss)
    
    print(f'Epoch {str(epoch + 1)} Train loss: {str(epoch_loss)}.')
          
print('Finished training')

Epoch: 1


IndexError: index 1 is out of bounds for dimension 0 with size 1